In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://beelike-winkingly-terra.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://beelike-winkingly-terra.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於台灣新竹縣新豐鄉的私立科技大學。學校創立於1966年，當時名為明新工業專科學校，後於1997年改制為明新技術學院，並最終在2002年升格為科技大學，定名為明新科技大學。

**辦學理念與願景：**
明新科技大學秉持校名「明新」之精義，取自《大學》「在明明德，在新民，在止於至善」，旨在闡揚人類與生俱來之德性與情操，以「深耕在地、放眼國際」為願景，目標是培養具實務經驗與人文素養的專業人才。學校的校訓為「堅毅、求新、創造」，並以成為「一流產業大學」作為最終發展願景。

**學術組織與特色：**
明新科技大學現設有半導體學院、工程學院、管理學院、民生學院、人文與設計學院、共同教育學院等六個學院，涵蓋20個學系、2個學位學程及11個碩士班。學校特別強調與產業鏈結，定位為「產業大學」，善用鄰近新竹科學園區與新竹工業區的地理優勢，積極推動產學合作。

**學校特色：**
*   **產業導向與實務教學：** 明新科大以培育產業所需實務人才為核心價值，強調實作教學，並有「星光熠熠」的國際星級師資陣容。在2022年《遠見》雜誌「企業最愛公私立技職科大調查」中，明新科大在四大領域中獲得兩項企業最愛，起薪排名私校第一，優於部分國立科大。
*   **前瞻產業佈局：** 學校緊密掌握世界與產業趨勢，鎖定半導體、AI、元宇宙、風電綠能等前瞻產業發展，並成立全台第一個「風力發電學士學位學程」。
*   **半導體人才培育：** 明新科大與國際封測大廠攜手，成立「半導體封裝測試產業聯盟」，為半導體產業聚落提供優質人才。
*   **跨域學習與國際視野：** 學校推動MUST（Multidisciplinary Learning, Universal Perspective, Sustainable Operations, Technological Innovation）四大育才特色，鼓勵學生跨域學習，並與海外大學簽訂雙聯學位合作，提升學生的國際移動力與全球學習視野。
*   **智慧科技應用：** 學校建置「永續智慧商務」教學與實習場域，導入生成式AI與AI專案應用，發展智慧零售、智慧金融、智慧製造與智慧商業等實驗室。同時，也將程式設計與人工智慧課程列為大一必修學

In [9]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長為**呂明峯教授**。

他於2025年2月1日正式上任，擔任明新科技大學的第11任校長。呂明峯校長具備豐富的半導體教育與產業實務經驗，並提出「四大核心模組」的校務治理策略，期望帶領明新科大開創技職教育新局。在接任校長之前，他曾任教於電子工程系，並歷任系主任、研發長、工學院院長、半導體學院院長以及產學長等職務。


In [10]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 儲存每個使用者的對話 session 和上傳的檔案
user_sessions = {}  # {user_id: {"chat": chat_object, "uploaded_file": gemini_file}}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:
        # 建立新的對話 session
        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    # 等待檔案處理完成
    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        # 有上傳檔案，使用 RAG 模式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        # 沒有上傳檔案，使用一般多輪對話
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 使用 RAG 或一般對話
                reply_text = query_with_rag(user_id, prompt)

                # 檢查是否有上傳檔案，加上提示
                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            # 清除使用者上傳的檔案
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 檢查檔案類型
        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            # 下載檔案
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            # 上傳到 Gemini
            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            # 儲存到使用者 session
            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[]}
INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:27:00] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[]}


INFO:__main__:Request body: {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"file","id":"594456925707173926","markAsReadToken":"YwL9iv7d4YhZyFrut4aVDSvCH6KQs9HoPAqgSyYF9KI6eN8cUAghcYRc9aNyEVbyBVABI4LKrazV1fUP5xvsnUnAxmHw_Ff1MKmbsP67KdNi_nubb5anA3vuWvqWpeJtgKMrBHGdFXAmi2FYUrYaUSIatcRTZmRMHvYs5L-uEXEIXPHRz2bNQD7oyxEO4pYhJY7wRtnyysNln4yfhHiSfw","fileName":"FAQ.txt","fileSize":24,"contentProvider":{"type":"line"}},"webhookEventId":"01KDSAJ6MZT7GEEVBG39VJWW0M","deliveryContext":{"isRedelivery":false},"timestamp":1767155243173,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"c6ecbf52d94e4e71bb26c942dc75ba82","mode":"active"}]}


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"file","id":"594456925707173926","markAsReadToken":"YwL9iv7d4YhZyFrut4aVDSvCH6KQs9HoPAqgSyYF9KI6eN8cUAghcYRc9aNyEVbyBVABI4LKrazV1fUP5xvsnUnAxmHw_Ff1MKmbsP67KdNi_nubb5anA3vuWvqWpeJtgKMrBHGdFXAmi2FYUrYaUSIatcRTZmRMHvYs5L-uEXEIXPHRz2bNQD7oyxEO4pYhJY7wRtnyysNln4yfhHiSfw","fileName":"FAQ.txt","fileSize":24,"contentProvider":{"type":"line"}},"webhookEventId":"01KDSAJ6MZT7GEEVBG39VJWW0M","deliveryContext":{"isRedelivery":false},"timestamp":1767155243173,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"c6ecbf52d94e4e71bb26c942dc75ba82","mode":"active"}]}
檔案已下載：/content/uploaded_files/FAQ.txt


...

INFO:__main__:Request body: {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"text","id":"594456984645271727","quoteToken":"V4_t10Z-r6ltcru6sxPLg80vPZvBBQ3lUCkjHVa8YBgLZfmO0zsQcgf-B4W-9E1FUO9RrlZHlsHWJ6I_WmBOhLjaIDirASa7CLRb2T5M9zcFOwJJX5VI91SigjHYRwQFvC1-bY7XGgGSt5_jcS_cOA","markAsReadToken":"leKLEfWMbsxVgPZigjhine_oVa0A8xwOwWxsbPngYcZIekfIvB-Pt6gY-zvabITS_uS-ATDQoMSyppO0yU9zOmEhY7diTfIqWCG193SHdoZx1m9X_FSV-FjhcYoxhTcW6wKySa7wyjXpOd-f7SruvJJ5o42lHi9Q8G4IC6J--1zuUtyvYk-wcqPfraMs8Th_xb6VtQpWfqf6L3Q2xZx4GA","text":"豬哥愛吃什麼"},"webhookEventId":"01KDSAK8X26N51P1A1GC1PJFY6","deliveryContext":{"isRedelivery":false},"timestamp":1767155278239,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"6b1d1b230d764fadb3c785308f9172e0","mode":"active"}]}


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"text","id":"594456984645271727","quoteToken":"V4_t10Z-r6ltcru6sxPLg80vPZvBBQ3lUCkjHVa8YBgLZfmO0zsQcgf-B4W-9E1FUO9RrlZHlsHWJ6I_WmBOhLjaIDirASa7CLRb2T5M9zcFOwJJX5VI91SigjHYRwQFvC1-bY7XGgGSt5_jcS_cOA","markAsReadToken":"leKLEfWMbsxVgPZigjhine_oVa0A8xwOwWxsbPngYcZIekfIvB-Pt6gY-zvabITS_uS-ATDQoMSyppO0yU9zOmEhY7diTfIqWCG193SHdoZx1m9X_FSV-FjhcYoxhTcW6wKySa7wyjXpOd-f7SruvJJ5o42lHi9Q8G4IC6J--1zuUtyvYk-wcqPfraMs8Th_xb6VtQpWfqf6L3Q2xZx4GA","text":"豬哥愛吃什麼"},"webhookEventId":"01KDSAK8X26N51P1A1GC1PJFY6","deliveryContext":{"isRedelivery":false},"timestamp":1767155278239,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"6b1d1b230d764fadb3c785308f9172e0","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:27:59] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"text","id":"594457008100082279","quoteToken":"HOywttuELfDuYQSf3XfeAHkjVuj8OguwjBl1YAciBUWUhGeoSkAmfGdsmpuA1tT5KQfCJxwYjzNea1nXXL-28CiSshJ9BWKuZZyW7jJSUBEEhRptf7mFwvZ2u3bIOHupI1mrtI-qWrla8M4rK1g-9w","markAsReadToken":"ZTXz7-wazwVQx3P45EG5AJcfhR-1Yy7dcYpMFhIxSXrIMuDBHsqaIs8XYknUGMu_fAehApM-FA32QeW6FYC71MqSo5jhPcRySE1Wzd_jzMYutgUFIx6Jgb_D3Dsob_sTnXAFE_ZjFRPgj_QZvNcWy-R3Hpo4aGzSkJk4sgNKSCmP4NdHAOZWOBnqDoc4rbqYXu2W3GrDAWja3SAj8jezNA","text":"AI 豬哥愛吃什麼"},"webhookEventId":"01KDSAKP4VMA69J80TT2TW547C","deliveryContext":{"isRedelivery":false},"timestamp":1767155292134,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"8243e511f7fa4914828802515cf347c1","mode":"active"}]}


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"text","id":"594457008100082279","quoteToken":"HOywttuELfDuYQSf3XfeAHkjVuj8OguwjBl1YAciBUWUhGeoSkAmfGdsmpuA1tT5KQfCJxwYjzNea1nXXL-28CiSshJ9BWKuZZyW7jJSUBEEhRptf7mFwvZ2u3bIOHupI1mrtI-qWrla8M4rK1g-9w","markAsReadToken":"ZTXz7-wazwVQx3P45EG5AJcfhR-1Yy7dcYpMFhIxSXrIMuDBHsqaIs8XYknUGMu_fAehApM-FA32QeW6FYC71MqSo5jhPcRySE1Wzd_jzMYutgUFIx6Jgb_D3Dsob_sTnXAFE_ZjFRPgj_QZvNcWy-R3Hpo4aGzSkJk4sgNKSCmP4NdHAOZWOBnqDoc4rbqYXu2W3GrDAWja3SAj8jezNA","text":"AI 豬哥愛吃什麼"},"webhookEventId":"01KDSAKP4VMA69J80TT2TW547C","deliveryContext":{"isRedelivery":false},"timestamp":1767155292134,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"8243e511f7fa4914828802515cf347c1","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:28:14] "POST / HTTP/1.1" 200 -
